<a href="https://colab.research.google.com/github/Valmik2004/Infosys-Springboard-Internship/blob/main/Day_7_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving fake_job_postings.csv to fake_job_postings (1).csv


In [2]:
# ---------------------------------------------
# Day 8: Saving Model and Vectorizer
# ---------------------------------------------

import pandas as pd
import joblib
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# --- Step 1: Load dataset ---
df = pd.read_csv('fake_job_postings.csv')

# --- Step 2: Check if 'clean_description' exists, else clean it ---
def clean_text(text):
    text = str(text).lower()                      # lowercase
    text = re.sub(r'<.*?>', ' ', text)            # remove HTML tags
    text = re.sub(r'[^a-z\s]', '', text)          # remove punctuation/numbers
    text = re.sub(r'\s+', ' ', text).strip()      # remove extra spaces
    return text

if 'clean_description' not in df.columns:
    print("🧹 Cleaning 'description' column...")
    df['clean_description'] = df['description'].fillna('').apply(clean_text)

# Drop missing or empty rows
df = df.dropna(subset=['clean_description'])
df = df[df['clean_description'].str.strip() != ""]

# --- Step 3: TF-IDF Vectorization ---
vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df['clean_description'])
y = df['fraudulent']

# --- Step 4: Train-Test Split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- Step 5: Train Model (Logistic Regression) ---
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# --- Step 6: Save Model and Vectorizer ---
joblib.dump(model, 'fake_job_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

print("✅ Model and Vectorizer Saved Successfully!")

# ---------------------------------------------
# Inference / Testing Saved Model
# ---------------------------------------------
# Load saved model and vectorizer
model = joblib.load('fake_job_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Example job descriptions
test_jobs = [
    "Work from home with high pay, no experience required! Apply immediately!",
    "We are hiring a software engineer with 2+ years of Python experience."
]

# Transform input text and predict
X_test_jobs = vectorizer.transform(test_jobs)
predictions = model.predict(X_test_jobs)

# Display predictions
for job, pred in zip(test_jobs, predictions):
    label = "Fake Job" if pred == 1 else "Real Job"
    print(f"\nJob: {job}\nPrediction: {label}")


🧹 Cleaning 'description' column...
✅ Model and Vectorizer Saved Successfully!

Job: Work from home with high pay, no experience required! Apply immediately!
Prediction: Real Job

Job: We are hiring a software engineer with 2+ years of Python experience.
Prediction: Real Job


In [3]:
!pip install flask pyngrok joblib

In [4]:
%%writefile app.py
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

model = joblib.load('fake_job_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

@app.route('/')
def home():
    return "✅ Fake Job Detection API is running on Colab!"

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    job_text = data.get('description', '') if data else ''
    if not job_text:
        return jsonify({"error": "No job description provided"}), 400
    X_input = vectorizer.transform([job_text])
    pred = model.predict(X_input)[0]
    label = "Fake Job" if pred == 1 else "Real Job"
    return jsonify({"prediction": label})

if __name__ == '__main__':
    app.run()


Writing app.py


In [5]:
from pyngrok import ngrok
ngrok.set_auth_token("354A7wIzV8oo6I3t1xYLDyPtsBH_4UifAjbTkMFA1pG6hLhgT")
public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)

!python app.py


🌍 Public URL: NgrokTunnel: "https://unbalanced-galen-holdable.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
